## Partner File Input
* Remove merge in each cell from top of Partner file. Do not worry about extra gray section rows. This scr will remove that.
* File must have title in row 0. Only one title.
  
### CPQ header format
* Product Code | Quantity | Unit Price |	Site
  
Author: Marlon Roa

In [ ]:
import pandas as pd    
import numpy as np
import scipy.stats as stats
import re
import datetime
from datetime import datetime, date, time, timedelta
#
from sys import argv
#
import os
import shutil
#

In [ ]:
#
Partner_COL_ST = 13 # After 1st 13 columns we should find the first quantity column "CANT."
MONTH_ST   = 3

# Ex Usage : num_tot = num_commas(po_df['ExtPrice'].sum())
def num_commas(number):
    return ("{:,.2f}".format(number))


In [ ]:
#
# =========================
# READ MODED FILE FROM Partner
# ========================

#
in_fname = "./Input/FORECAST_INPUT.xlsx"
xls = pd.ExcelFile(in_fname) #
Partner_df = pd.read_excel(xls)
#
price_df = pd.read_csv("./Input/Partner_pricing.csv")
#
# =========================
# CLEAN FILES
# ========================
#
Partner_df = Partner_df[Partner_df['NOMBRE'].notnull()]
Partner_df.reset_index(drop=True, inplace=True)
#
# =========================
# DATA TRANSFORMATION
# ========================
#
# I: Standard rename of columns, uppercase Product in both files.
#
Partner_df.rename(columns={'CODIGO':'Product', 'NOMBRE':'Descr', 'PRECIO UNITARIO':'TP'}, inplace=True)
Partner_df['Product'] = Partner_df['Product'].str.upper()
#
Partner_df.to_excel("./Output/fyi_Partner_df.xlsx", sheet_name='Sheet')
#
price_df['Product Code'] = price_df['Product Code'].str.upper()
#
# I: Prepare month arrays for file production.
#
month_l = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep','Oct', 'Nov', 'Dec']
sel_m = month_l[MONTH_ST:]
#
print("I: Months selected to create forecast:", sel_m)
#


In [ ]:
# =========================
# EXECUTE FUNCTION
# ========================
#
new_df = pd.DataFrame()
#
print ("I: Running for loop for Partner file w/ Commit/Upside per month")
for idx in list(Partner_df.index):
    pon  = Partner_df['Product'].loc[idx]
    tp   = Partner_df['TP'].loc[idx] #TP  
    n_ix = Partner_COL_ST
    #
    for m in sel_m:
        qty1 = Partner_df.iloc[idx, n_ix+1] #COMMIT
        qty2 = Partner_df.iloc[idx, n_ix+2] #UPSIDE              
        #
        n_ix += 4
        #
        if (np.isnan(qty1)): qty1 = 0
        if (np.isnan(qty2)): qty2 = 0
        if (np.isnan(tp)):   tp   = 0
        #
        new_row = pd.Series(data={'Product Code':pon, 'qty_c':qty1, 'qty_u':qty2, 'oPrice':tp, 'month':m}, name='I')
        new_df = new_df.append(new_row, ignore_index=False)
    #./. eo for sel_m
    #
#./.
#
print ("I: Add pricing per month and produce CPQ file. Report total")
#
fcast_t = ['Commit', 'Upside']
qty_t = {'Commit':'qty_c', 'Upside':'qty_u'}
#
total_Partner = {'Commit':0, 'Upside':0}
total_inf = {'Commit':0, 'Upside':0}
#
for m in sel_m:
    #
    # I: Filter out the month & add the Our price for each PON.
    tmp_df = new_df[new_df['month'] == m]
    total_df = pd.merge(tmp_df, price_df, how="left", on='Product Code')    
    #
    for i in fcast_t:  
        #
        # I: Select commit or upside category where qty != 0 to create file for it
        sub_df = total_df[total_df[qty_t[i]] != 0].copy()
        #
        # I: Calculate totals for the category so it can be reported
        sub_df['inf_t'] = sub_df['Unit Price'] * sub_df[qty_t[i]]
        sub_df['Partner_t'] = sub_df['oPrice'] * sub_df[qty_t[i]]        
        total_Partner[i] += sub_df['Partner_t'].sum()
        total_inf[i] += sub_df['inf_t'].sum()
        #
        num1 = num_commas(sub_df['Partner_t'].sum())
        num2 = num_commas(sub_df['inf_t'].sum())
        print("\tCreating ", i, " CPQ for month: ", m, " Total Partner: ", num1, " Total INF: ", num2)        
        #
        sub_df = sub_df[sub_df['Product Code'].notnull()] # Rm the sums
        sub_df = sub_df[['Product Code', qty_t[i], 'Unit Price', 'Site']]
        sub_df.rename(columns={qty_t[i]:'Quantity'}, inplace=True)
        if (sub_df['Unit Price'].isnull().values.any() == True): print("\t**WARN**(",m,") Missing Unit price\n")
        file_o_txt = "./Output/Partner_"+m+"_"+i+".csv"
        sub_df.to_csv(file_o_txt, index=False)
    # ./. Eo for i
    #
# ./. eo for m
#
num1 = num_commas(total_Partner['Commit'])
num2 = num_commas(total_Partner['Upside'])
print ("Partner_Total: {Commit: ", num1, " Upside: ", num2 , "} ,", num_commas(total_Partner['Commit']+total_Partner['Upside']))
num1 = num_commas(total_inf['Commit'])
num2 = num_commas(total_inf['Upside'])
print ("INF_Total: {Commit: ", num1, " Upside: ", num2, "} ,", num_commas(total_inf['Commit']+total_inf['Upside']))
    

